In [1]:
import numpy as np
import torch
import pickle
import time
import os
%matplotlib inline
import matplotlib.pyplot as plt


In [2]:
if not os.path.isfile('superpixels.zip'):
    print('downloading..')
    !curl https://www.dropbox.com/s/y2qwa77a0fxem47/superpixels.zip?dl=1 -o superpixels.zip -J -L -k
    !unzip superpixels.zip -d ../
    # !tar -xvf superpixels.zip -C ../
else:
    print('File already downloaded')

downloading..
zsh:1: no matches found: https://www.dropbox.com/s/y2qwa77a0fxem47/superpixels.zip?dl=1
unzip:  cannot find or open superpixels.zip, superpixels.zip.zip or superpixels.zip.ZIP.


In [3]:
import os

print(os.getcwd())

/Users/tuyiwen/Desktop/gli/datasets/cifar


In [4]:
import pickle

%load_ext autoreload
%autoreload 2

from data.superpixels import SuperPixDatasetDGL 

from data.data import LoadData
from torch.utils.data import DataLoader
from data.superpixels import SuperPixDataset

Using backend: pytorch


In [5]:
start = time.time()

DATASET_NAME = 'CIFAR10'
dataset = SuperPixDatasetDGL(DATASET_NAME) 

print('Time (sec):',time.time() - start) # 636s=10min

Adj matrix defined from super-pixel locations (only)
preparing 10000 graphs for the TEST set...


/Users/tuyiwen/Library/Python/3.8/lib/python/site-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


preparing 50000 graphs for the TRAIN set...
[I] Data load time: 2126.8937s
Time (sec): 2126.9006509780884


In [6]:
print(len(dataset.train))
print(len(dataset.val))
print(len(dataset.test))

print(dataset.train[0])
print(dataset.val[0])
print(dataset.test[0])

45000
5000
10000
(Graph(num_nodes=110, num_edges=880,
      ndata_schemes={'feat': Scheme(shape=(5,), dtype=torch.float16)}
      edata_schemes={'feat': Scheme(shape=(1,), dtype=torch.float16)}), tensor(6))
(Graph(num_nodes=123, num_edges=984,
      ndata_schemes={'feat': Scheme(shape=(5,), dtype=torch.float16)}
      edata_schemes={'feat': Scheme(shape=(1,), dtype=torch.float16)}), tensor(6))
(Graph(num_nodes=118, num_edges=944,
      ndata_schemes={'feat': Scheme(shape=(5,), dtype=torch.float16)}
      edata_schemes={'feat': Scheme(shape=(1,), dtype=torch.float16)}), tensor(3))


In [7]:
train_idx=np.arange(0,45000,1)
valid_idx=np.arange(45000,50000,1)
test_idx=np.arange(50000,60000,1)

In [8]:
task_data = {
    "train": train_idx,
    "val": valid_idx,
    "test": test_idx
}
np.savez_compressed("graph_classification_1.npz", **task_data)

In [9]:
from tqdm import tqdm
node_list = []
labels = []
edges = []
edge_feats = []
node_feats = []
num_nodes = 0
for g, label in tqdm(dataset.train):
    node_list.append(np.arange(g.num_nodes()) + num_nodes)  # All the nodes are considered in a single graph
    
    labels.append(label)
    edges.append(np.stack([g.edges()[0],g.edges()[1]]).T + num_nodes)
    edge_feats.append(np.array(g.edata['feat']))
    node_feats.append(np.array(g.ndata['feat']))
    num_nodes += g.num_nodes()
for g, label in tqdm(dataset.val):
    node_list.append(np.arange(g.num_nodes()) + num_nodes)  # All the nodes are considered in a single graph
    labels.append(label)
    edges.append(np.stack([g.edges()[0],g.edges()[1]]).T + num_nodes)
    edge_feats.append(np.array(g.edata['feat']))
    node_feats.append(np.array(g.ndata['feat']))
    num_nodes += g.num_nodes()
for g, label in tqdm(dataset.test):
    node_list.append(np.arange(g.num_nodes()) + num_nodes)  # All the nodes are considered in a single graph


    labels.append(label)
    edges.append(np.stack([g.edges()[0],g.edges()[1]]).T + num_nodes)
    edge_feats.append(np.array(g.edata['feat']))
    node_feats.append(np.array(g.ndata['feat']))
    num_nodes += g.num_nodes()

100%|██████████| 10000/10000 [00:03<00:00, 2555.88it/s]



_edges = np.concatenate(edges, axis=0)
_labels = np.stack(labels).squeeze()
_edge_feats = np.concatenate(edge_feats)
_node_feats = np.concatenate(node_feats)

In [11]:
import scipy.sparse as sparse
_node_list = sparse.csr_matrix((60000, num_nodes))

for i, indices in enumerate(node_list):
    _node_list[i, indices] = 1

In [14]:
data = {
    "node_feats": _node_feats,
    "graph_class": _labels,
    "edge": _edges,
    "edge_feats": _edge_feats,
    "node_list": _node_list,
}

In [13]:

_edges = np.concatenate(edges, axis=0)
_labels = np.stack(labels).squeeze()
_edge_feats = np.concatenate(edge_feats)
_node_feats = np.concatenate(node_feats)

In [15]:
np.savez_compressed("mnist.npz", **data)